
# Análisis de grupos con MNE-Python


El objetivo de este tutorial es mostrar cómo hacer análisis de grupos con MNE-Python.

    Autores: Brigitte Aguilar, Sofía Poux, Elizabeth Young
    Modificado de: Britta Westner, Alexandre Gramfort, Denis A. Engemann, Mainak Jas, Hicham Janati

    Licencia: BSD (claúsula 3)
   

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')
import os
import numpy as np
import mne

mne.set_log_level('error')

# Cambie la siguiente ruta a donde está la carpeta 'extra_data_mne' dentro de la carpeta 'Datos' en su disco:
extra_path = os.path.expanduser('~/Documents/MNE-projects/mne_tutorial_GardenOroVerde/Datos/extra_data_mne')
evokeds_path = os.path.join(extra_path, 'group_analysis/')

Comprobemos si la configuración de la ruta es correcta y si los datos están ahí:

In [ ]:
 ls $evokeds_path

## Datasets evocados

Tenemos los datos evocados para todos los participantes. Sin embargo, los datos de un participante (dataset # 10) tienen problemas con los triggers que necesitan ser arreglados antes de trabajar con ellos, por lo tanto, por ahora, vamos a descartar los datos de ese participante:

In [ ]:
datasets = ['sub-%02d' % ii for ii in range(1, 17) if ii not in [10]]
datasets

## Veamos nuestros datasets

Podemos ver los datasets uno por uno:

In [ ]:
subject = 'sub-02'
fname = os.path.join(evokeds_path, ('%s_list-ave.fif' % subject))
evokeds = mne.read_evokeds(fname, verbose=False)
evokeds

¿Qué son los objetos evoked? Los objetos evocados o evoked normalmente almacenan señales EEG o MEG que se han promediado a lo largo de múltiples épocas.

In [ ]:
%matplotlib qt
evokeds[0].plot_joint()

Los join plots combinan gráficos de mariposas con topografías y proporcionan un excelente primer vistazo a los datos evocados; de forma predeterminada, las topografías se colocarán automáticamente según los picos encontrados. Aquí trazamos la condición "famous"; si no se especifican tipos de canales (con picks), obtenemos una gráfica separada para cada tipo de sensor.

También podemos recorrer todos los datasets para obtener una descripción general y de esta manera observar si hay algún dataset poblemático:

In [ ]:
%matplotlib inline
ch_type = 'eeg'  # elegimos los canales de EEG
conditions = ['famous', 'unfamiliar', 'scrambled']

f, axes = plt.subplots(4, 4, figsize=(13, 9), sharex=True, sharey=True)

for ax, subject in zip(axes.ravel(), datasets):
    evokeds_dict = dict()
    fname = os.path.join(evokeds_path, ('%s_list-ave.fif' % subject))
    evokeds = mne.read_evokeds(fname)
    evokeds = [ev for ev in evokeds if ev.comment in conditions]
    for condition, ev in zip(conditions, evokeds):
        evokeds_dict[condition] = ev.crop(tmin=-0.3, tmax=0.6)
    mne.viz.plot_compare_evokeds(evokeds_dict, picks=ch_type, show=False,
                                 axes=ax, title=subject)

plt.tight_layout()

¿Qué significa GFP? Global Field Power (GFP) es una medida de la (no)uniformidad del campo electromagnético en los sensores. Normalmente se calcula como la desviación estándar de los valores del sensor en cada momento. Por lo tanto, es una serie temporal unidimensional que captura la variabilidad espacial de la señal en las ubicaciones de los sensores.

<div class='alert alert-success'>
    <b>EJERCICIO</b>:
     <ul>
      <li>Realizar los mismos gráficos GFP para el caso de magnetómetros</li>
      <li>¿Observas algún dataset problemático?</li>
    </ul>
</div>

## Leer todos los datos y calcular contraste

Ahora, leeremos todos los datasets y los apilaremos en _una lista_:

In [ ]:
evokeds_list = []

for subject in datasets:
    fname = os.path.join(evokeds_path, ('%s_list-ave.fif' % subject))
    evokeds = mne.read_evokeds(fname)
    evokeds = [ev for ev in evokeds if ev.comment in ['famous', 'scrambled']]
    evokeds_list.append(evokeds)

evokeds_list

A partir de aquí, podemos obtener el contraste, por ejemplo entre _famous_ y _scrambled_. Podemos visualizar gráficamente este contraste:

In [ ]:
contrast_list = []
f, axes = plt.subplots(4, 4, figsize=(13, 9), sharex=True, sharey=True)

for ax, subject, evokeds in zip(axes.ravel(), datasets, evokeds_list):
    contrast = mne.combine_evoked(evokeds, weights=[0.5, -0.5]).crop(-0.3, 0.5)
    contrast.comment = 'contrast'
    contrast_list.append(contrast)
    mne.viz.plot_compare_evokeds(contrast, picks=ch_type, show=False,
                                 axes=ax, title=subject)
plt.tight_layout()

In [ ]:
contrast_list

### Una mirada al promedio total (grand average)

Los _Grand averages_ se obtienen mediante el promedio de los datos del espacio de sensores.

In [ ]:
mne.grand_average?

In [12]:
%matplotlib qt
evoked_gave = mne.grand_average(contrast_list)
evoked_gave.plot_joint();

## Algunas estadísticas - test de permutación de clústers para un canal

Comenzaremos utilizando un único canal, EEG065. 

Realizaremos un test de permutación de clústers de nuestro contraste contra 0.
Comencemos con la configuración:

In [13]:
# importar paquetes para las estadísticas

import scipy as sp
from mne.stats import permutation_cluster_1samp_test

Llevemos nuestros datos a una arreglo de numpy para que podamos pasarlo como parámetro a la función de estadísticas:

In [14]:
channel = 'EEG065'
ch_idx = contrast.ch_names.index(channel)
data = np.array([c.data[ch_idx] for c in contrast_list])

Ahora, configuremos algunos parámetros para nuestro test:

In [15]:
n_jobs = 1  # Para el número de trabajos paralelos, es posible que desees establecerlo en 1
n_permutations = 5000  # número de permutaciones a correr

# Especificar la adyacencia de los puntos en los datos: 
# Aquí, no se necesita una adyacencia especial porque es solo un canal y MNE 
# simplemente asumirá una cuadrículas regular, lo cual es adecuado para nuestra dimensión de
adjacency = None  

tail = 0.  # hacemos un test de dos colas (two-sided test)

Con eso resuelto, establezcamos los umbrales de los clústers:

In [16]:
# establecer el valor p que queremos utilizar para realizar nuestra prueba
p_value = 0.05

#Ahora, calculemos el umbral de valor t (threshold) para un valor p dado
# y un tipo de prueba (bilateral o unilateral) el cual
# necesitamos pasarlo a la función de estadísticas. 
# Tené en cuenta que esto cambia para una prueba de una cola.
deg_of_free = len(data) - 1
threshold = sp.stats.t.ppf(1 - p_value/ (1 + (tail == 0)), deg_of_free)

Ahora tenemos todo para poder realizar nuestras estadísticas:

In [17]:
cluster_stats = permutation_cluster_1samp_test(
    data, 
    threshold=threshold, 
    n_jobs=n_jobs, 
    verbose=True, 
    tail=tail,
    adjacency=adjacency,
    n_permutations=n_permutations, 
    seed=42)

T_obs, clusters, cluster_p_values, _ = cluster_stats

stat_fun(H1): min=-6.783906 max=1.635025
Running initial clustering …
Found 2 clusters


  0%|          | Permuting : 0/4999 [00:00<?,       ?it/s]

#### Visualizamos los resultados para el test de un sólo canal

In [18]:
# configuramos la figura:
fig, axes = plt.subplots(2, sharex=True)

# En el primer eje: graficar los datos (contraste promediado en todos los conjuntos de datos)
ax = axes[0]
ax.plot(contrast.times, data.mean(axis=0), label='ERP Contrast')
ax.set(title='Channel : ' + channel, ylabel='fT/cm')
ax.legend()

# en el segundo eje:
ax = axes[1]
# enumerar a través de los clústers
for ii, cluster_ii in enumerate(clusters):
    c = cluster_ii[0]

    # verificar si el valor p coincidente es menor que el umbral:
    if cluster_p_values[ii] < p_value:
        # Si es así, marcar el período de tiempo:
        h1 = ax.axvspan(contrast.times[c[0]], 
                        contrast.times[c[-1] - 1],
                        color='r', alpha=0.3)

# graficamos los valores t 
hf = ax.plot(contrast.times, T_obs, 'g')

# establecemos leyenda, ejes y graficamos
ax.legend((h1,), (u'p < %s' % p_value,), loc='upper right', ncol=1)
ax.set(xlabel='Time (ms)', ylabel='T-values',
       ylim=[-10., 10.], xlim=contrast.times[[0, -1]])
fig.tight_layout(pad=0.5)
plt.show()

## Test de permutación de clústers a través del tiempo y de los sensores

Ahora, elijamos un sensor en particular y ejecutemos la permutación de clústers a través de los _magnetómetros_: 

In [19]:
# Aquí podemos utilizar una función conveniente para datos espacio-temporales:
from mne.stats import spatio_temporal_cluster_1samp_test

In [ ]:
# elijamos los magnetómetros:
ch_type = 'mag'

# Acá otra vez tenemos que llevar los datos a un array de numpy
data = np.array([c.copy().pick_types(meg=ch_type).data
                 for c in contrast_list])

data.shape

La función de clústering nos dice:
```
X : array, shape (n_observations, p[, q], n_vertices)
    Los datos que se van a agrupar. La primera dimensión debería corresponder a la diferencia entre muestras emparejadas  (observaciones) en dos condiciones. La segunda y, opcionalmente, la tercera dimensión corresponden a los datos de tiempo o tiempo-frecuencia. Y la última dimensión debería ser espacial.
```

In [ ]:
# Asegurémonos que la dimensión espacial es la última:
data = np.transpose(data, (0, 2, 1)) 
data.shape

Configuramos nuestros parámetros nuevamente. Esta vez debemos prestar mucha atención a la adyacencia!

In [22]:
# umbrales de clusters y nro de colas
tail = 0.  # para test de dos colas

# configuramos el umbral de clústers
deg_of_free = len(data) - 1
threshold = sp.stats.t.ppf(1 - p_value/ (1 + (tail == 0)), deg_of_free)

# Crear una triangulación entre las ubicaciones de los sensores MEG 
# para utilizar como información de adyacencia en las estadísticas:
adjacency = mne.channels.find_ch_adjacency(contrast.info, ch_type)[0]

In [ ]:
cluster_stats = spatio_temporal_cluster_1samp_test(
    data, 
    threshold=threshold, 
    n_jobs=2, 
    verbose=True, 
    tail=tail,
    adjacency=adjacency, 
    out_type='indices',
    check_disjoint=True, 
    seed=42)

In [ ]:
# Modificamos la salida para que sea más fácil de graficar:
T_obs, clusters, p_values, _ = cluster_stats
good_cluster_inds = np.where(p_values < 0.05)[0]

print("Good clusters: %i" % len(good_cluster_inds))

#### Visualizamos los clústers espacio-temporales

In [25]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mne.viz import plot_topomap

# algunas configuraciones para la graficación
colors = 'r', 'steelblue'
linestyles = '-', '--'

# encontramos los sensores relevantes
pos = mne.find_layout(contrast.info, ch_type=ch_type).pos

T_obs_max = 5.
T_obs_min = -T_obs_max

# blucle sobre los sensores relevantes
for i_clu, clu_idx in enumerate(good_cluster_inds):

    # desempaquetar la información de clústers, obtener índices únicos por clúster
    time_inds, space_inds = np.squeeze(clusters[clu_idx])
    ch_inds = np.unique(space_inds)
    time_inds = np.unique(time_inds)

    # obtener la topografía para estadísticas y promediar a lo largo del tiempo
    T_obs_map = T_obs[time_inds, ...].mean(axis=0)

    # obtener señales en sensores significativos y promediar a lo largo de los sensores
    signals = data[..., ch_inds].mean(axis=-1)
    sig_times = contrast.times[time_inds]

    # crear una máscara espacial
    mask = np.zeros((T_obs_map.shape[0], 1), dtype=bool)
    mask[ch_inds, :] = True

    # inicializar figura
    fig, ax_topo = plt.subplots(1, 1, figsize=(7, 2.))

    # marcar los sensores en el clúster
    mask_params = dict(marker='.', markerfacecolor='k', markersize=2)

    # graficar el promedio de la prueba estadística y marcar los sensores significativos
    sel = mne.pick_types(contrast.info, meg=ch_type)
    info = mne.pick_info(contrast.info, sel)
    image, _ = plot_topomap(T_obs_map, info, ch_type=ch_type, mask=mask, 
                            axes=ax_topo, sensors=False,
                            mask_params=mask_params,
                            vlim=(T_obs_min,T_obs_max),
                            show=False)

    # mostrar en una misma figura la imagen y barra de colores
    divider = make_axes_locatable(ax_topo)

    # añadir los ejes para la barra de colores y la barra de colores 
    ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
    plt.colorbar(image, cax=ax_colorbar, format='%0.1f')

    # etiquetar la topografía
    ax_topo.set_xlabel('Averaged t-map\n({:0.2f} - {:0.2f} ms)'.format(
        *sig_times[[0, -1]]
    ))

    # agregar un nuevo eje para las series de tiempo y graficar las series de tiempo
    ax_signals = divider.append_axes('right', size='300%', pad=1.2)
    for signal, name, col, ls in zip(signals, ['Contrast'], colors,
                                     linestyles):
        ax_signals.plot(contrast.times, signal, color=col,
                        linestyle=ls, label=name)

    # marcar el inicio del estímulo
    ax_signals.axvline(0, color='k', linestyle=':', label='stimulus onset')

    # ajustar y etiquetar ejes
    ax_signals.set_xlim([contrast.times[0], contrast.times[-1]])
    ax_signals.set_xlabel('Time [s]')
    ax_signals.set_ylabel('Amplitude')

    # graficar el rango de tiempo significativo en la serie de tiempo
    ymin, ymax = ax_signals.get_ylim()
    ax_signals.fill_betweenx((ymin, ymax), sig_times[0], sig_times[-1],
                             color='orange', alpha=0.3)
    ax_signals.legend(loc='lower right')
    title = 'Cluster #{0} (p < {1:0.3f})'.format(i_clu + 1, p_values[clu_idx])
    ax_signals.set(ylim=[ymin, ymax], title=title)

    # limpiamos la figura un poco
    fig.tight_layout(pad=0.5, w_pad=0)
    fig.subplots_adjust(bottom=.05)

plt.show()

#### Lecturas adicionales y créditos:

El código de esta notebook está fuertemente inspirado en: https://github.com/mne-tools/mne-biomag-group-demo/tree/master/scripts/results/statistics

Podés encontrar más información sobre las estadísticas a nivel de sensor aquí: https://mne.tools/stable/auto_tutorials/stats-sensor-space/index.html

Si deseas hacer estadísticas en el espacio de las fuentes, aquí más información: https://mne.tools/stable/auto_tutorials/stats-source-space/index.html